# SoccerNetPlus
by AiJing Wu, Michael Vanden Heuvel

## Environment settings:
1. Create a directory/folder named "SoccerNetPlus" and put everything in it
2. Runtime -> Change runtime type -> GPU
3. Mount the drive to colab (with the next code snippet)

In [4]:
# interact with drive files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Important variables
max_allowed_video_length = 10
parent_dir = '/content/gdrive/My Drive/SoccerNetPlus/'
download_dir = '/content/gdrive/My Drive/SoccerNetPlus/downloaded_videos'
output_dir = '/content/gdrive/My Drive/SoccerNetPlus/video_output'
anvil_token_dir = "/content/gdrive/MyDrive/SoccerNetPlus/anvil_key.txt"

In [6]:
%cd $parent_dir

/content/gdrive/MyDrive/Classes/Year 5/SoccerNetPlus


## Imports

In [7]:
# download all dependencies
!pip install -r dependency-requirements.txt
!pip install anvil-uplink
!pip install youtube_dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.5 MB 1.9 kB/s 
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.1.26 (from versions: 3.4.0.14, 3.4.8.29, 3.4.9.31, 3.4.9.33, 3.4.10.35, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.14.51, 3.4.14.53, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.1.2.30, 4.2.0.32, 4.2.0.34, 4.3.0.36, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.2.52, 4.5.2.54, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66)
ERROR: No matching distribution found for opencv-python==4.1.1.26
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 6.3 MB/s 
     |████████████████████████████████| 51 kB 230 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 s

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 23.4 MB/s 


In [8]:
# import all libraries
from IPython.display import YouTubeVideo, display
import requests
import os, subprocess
import cv2
import urllib.parse as urlparse
import datetime

### Anvil Setup

In [9]:
# link with Anvil frontend
import anvil.server
with open(anvil_token_dir, "r") as f:
  anvil.server.connect(f.read())

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


## Model Calls

### Detection and 2D Birdeye Transform
with Yolov3 & OpenCV, all running in SoccerNetPlus/yolov3_opencv

In [10]:
@anvil.server.callable
def detection_transformation(input_video_filename):
    os.chdir(os.path.join(parent_dir, "yolov3_opencv"))

    input_path = os.path.join(download_dir, input_video_filename)
    # print(input_path)

    output_path_detection = os.path.join(output_dir, "detection-"+input_video_filename)
    output_path_bird = os.path.join(output_dir, "birds-eye-"+input_video_filename)
    # print(output_path_detection)
    
    # if not os.path.isfile(output_path_detection) and not os.path.isfile(output_path_bird):

    # Command: birdeye_view.py path-to-input-video path-to-output-video-detection-part path-to-output-video-birdeye-part
    # subprocess.call("python birdeye_view.py ../video_input/test-video.mp4 ../video_output/detection.mp4 ../video_output/birdeye.mp4", shell=True)
    print()
    subprocess.call(f"python birdeye_view.py {input_path} {output_path_bird} {output_path_detection}", shell=True)
    os.chdir(parent_dir)
    return output_path_detection

In [24]:
# detection_transformation("test-video-Clip 2.mp4")

### Tracking and Statistics Display
with Yolov3 & DeepSort, all running in SoccerNetPlus/yolov3_deepsort

In [12]:
@anvil.server.callable
def tracking_statistics(input_video_filename):
    os.chdir(os.path.join(parent_dir, "yolov3_deepsort"))
    
    input_path = os.path.join(download_dir, input_video_filename)
    output_path_tracking = os.path.join(output_dir, "tracking-"+input_video_filename)

    # if not os.path.isfile(output_path_detection):

    subprocess.call("python load_weights.py", shell=True)

    # Command: object_tracker.py --video path-to-input-video --output path-to-output-video
    subprocess.call(f"python object_tracker.py --video {input_path} --output {output_path_tracking}", shell=True)
    os.chdir(parent_dir)
    return output_path_tracking

In [13]:
# tracking_statistics("2022-12-04_01.54.00_3loq22TxSc")

## UI Code

### Populate Example Video Dropdown

In [14]:
@anvil.server.callable
def populate_example_video_dropdown(clip_duration=max_allowed_video_length):
  global video_input
  video_input = {}
  dir = os.path.join(parent_dir, "video_input")
  videos = [x for x in os.listdir(dir) if x.endswith(".mp4")]
  video_durations = []
  for video in videos:
    video = os.path.join(dir,video)
    vidcapture = cv2.VideoCapture(video)
    fps = vidcapture.get(cv2.CAP_PROP_FPS)
    totalNoFrames = vidcapture.get(cv2.CAP_PROP_FRAME_COUNT)
    video_durations.append(int(totalNoFrames / fps))

  for i, v in enumerate(zip(videos,video_durations)):
    video, duration = v
    # print(f"Duration: {duration}")
    # print(range(0,clip_duration,duration))
    # print(list(range(0,clip_duration,duration)))
    for j,s in enumerate(range(0,duration,clip_duration)):
      # print(f"Clip {((i+1)*(j+1))}, dur: {duration}")
      video_input[f"Clip {((i+1)*(j+1))}"] = {"video_name": video, "path": os.path.join(dir, video), "start":s, "clip_duration": min(clip_duration,duration-clip_duration*(j))}
  # print(duration)
  # print(video_input)
  return list(video_input.keys())
  
# populate_example_video_dropdown()
# video_input

### Get image from video

In [15]:
def get_screenshot(video_filename, screenshot_frame_num=10):
  vidcap = cv2.VideoCapture(video_filename)
  success,image = vidcap.read()
  count = 0
  while success and count < screenshot_frame_num:
    success,image = vidcap.read()
    count += 1  

  return image

# # import cv2
# vidcap = cv2.VideoCapture('big_buck_bunny_720p_5mb.mp4')
# success,image = vidcap.read()
# count = 0
# while success:
#   cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
#   success,image = vidcap.read()
#   print('Read a new frame: ', success)
#   count += 1

print(type(get_screenshot("/content/gdrive/My Drive/SoccerNetPlus/downloaded_videos/test-video-Clip 2.mp4")))

<class 'numpy.ndarray'>


## YouTube Helper Functions

In [16]:
@anvil.server.callable
def get_video_id(value):
    """
    Examples:
    - http://youtu.be/Ns8NvPPHX5Y
    - http://www.youtube.com/watch?v=Ns8NvPPHX5Y&feature=feedu
    - http://www.youtube.com/embed/Ns8NvPPHX5Y
    - http://www.youtube.com/v/Ns8NvPPHX5Y?version=3&amp;hl=en_US
    """
    query = urlparse.urlparse(value)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = urlparse.parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    # fail?
    return None

In [17]:
# Source: https://stackoverflow.com/questions/63325908/how-do-i-check-if-a-youtube-video-url-is-valid-or-not-in-python
@anvil.server.callable
def check_video_url(video_id):
    checker_url = "https://www.youtube.com/oembed?url=http://www.youtube.com/watch?v="
    video_url = checker_url + video_id

    request = requests.get(video_url)

    return request.status_code == 200
# check_video_url("SB-qEYVdvXA")

In [18]:
# Source: https://stackoverflow.com/questions/63325908/how-do-i-check-if-a-youtube-video-url-is-valid-or-not-in-python
@anvil.server.callable
def get_video_thumbnail(video_id):
    checker_url = "https://www.youtube.com/oembed?url=http://www.youtube.com/watch?v="
    video_url = checker_url + video_id

    request = requests.get(video_url)

    return request.json()["thumbnail_url"]
# check_video_url("SB-qEYVdvXA")

In [19]:
def get_video_duration(video_id):
  cap = cv2.VideoCapture("./video.mp4") # Issue is getting the video to check its duration
  fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  duration = frame_count/fps
  return duration

In [20]:
def check_video_duration(video_id, start_time):
  return start_time < get_video_duration(video_id)

In [21]:
@anvil.server.callable
def download_youtube_clip(video_id, start_time, duration=max_allowed_video_length):
  video_url = "https://www.youtube.com/watch?v=" + video_id

  t = datetime.timezone(datetime.timedelta(hours=-5)) # CT Timezone offset -5 from UTC (varies based on DST)
  current_time = datetime.datetime.now(tz=t).strftime("%Y-%m-%d_%H.%M.%S") # Current time and video url provides a unique filename
  
  output_video_filename = current_time + video_id + ".mp4"

  command = f"ffmpeg $(youtube-dl -g '{video_url}' | sed 's/.*/-ss {start_time} -i &/') -t {duration} -c copy {output_video_filename}"
  
  os.chdir(download_dir)
  r_code = os.system(command)
  os.chdir(parent_dir)
  
  return (r_code == 0, output_video_filename)

## Example Video Helper Functions


In [22]:
@anvil.server.callable
def clip_example_video(clip_key):
  """
  References:
  https://superuser.com/questions/138331/using-ffmpeg-to-cut-up-video
  https://stackoverflow.com/questions/18444194/cutting-the-videos-based-on-start-and-end-time-using-ffmpeg
  """
  filename = '.'.join(video_input[clip_key]["video_name"].split(".")[:-1])
  extension = video_input[clip_key]["video_name"].split(".")[-1]
  output_filename = f"{filename}-{clip_key}.{extension}"
  output = os.path.join(download_dir, output_filename)

  if os.path.isfile(output):
    print("Skipped because file already exists!")
    return (True, output_filename)

  # command = f"ffmpeg -i '{video_input[clip_key]['path']}' -ss {video_input[clip_key]['start']} -to {video_input[clip_key]['start']+video_input[clip_key]['clip_duration']} -c copy '{output}'"
  # command = f"ffmpeg -ss {video_input[clip_key]['start']} -i '{video_input[clip_key]['path']}' -c copy '{output}' -t {video_input[clip_key]['clip_duration']}"
  
  # Ordering for ffmpeg apparently matters a lot. There is probably a more quicker way for this command to run, but this is what I was able to find so that
  # the videos were split correctly. Issues are likely related to .mp4 format and keyframes?
  command = f"ffmpeg -ss {video_input[clip_key]['start']} -i '{video_input[clip_key]['path']}' -to {video_input[clip_key]['clip_duration']} '{output}' "

  os.chdir(download_dir)
  r_code = os.system(command)
  os.chdir(parent_dir)
  
  return (r_code == 0, output_filename)

populate_example_video_dropdown()
clip_example_video("Clip 1")


Skipped because file already exists!


(True, 'test-video-Clip 1.mp4')

In [23]:
def clip_all_example_videos():
  for key in video_input:
    print(f"{key}: start: {video_input[key]['start']} Duration:{video_input[key]['clip_duration']}")
    print(clip_example_video(key))
    
clip_all_example_videos()

Clip 1: start: 0 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 1.mp4')
Clip 2: start: 10 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 2.mp4')
Clip 3: start: 20 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 3.mp4')
Clip 4: start: 30 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 4.mp4')
Clip 5: start: 40 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 5.mp4')
Clip 6: start: 50 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 6.mp4')
Clip 7: start: 60 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 7.mp4')
Clip 8: start: 70 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 8.mp4')
Clip 9: start: 80 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 9.mp4')
Clip 10: start: 90 Duration:10
Skipped because file already exists!
(True, 'test-video-Clip 10.mp4')
Clip 11: 

# Anvil Wait

In [ ]:
anvil.server.wait_forever()